In [26]:
import pandas as pd
import numpy as np
import requests

# Requested personal data from spotify and loading this data
df = pd.read_csv("spotifydata_harneet30.csv")
# dropping unnecessary columns 
df = df.drop(["Unnamed: 4","Unnamed: 5","Unnamed: 6"], axis =1)
df

,endTime,artistName,trackName,msPlayed
0,2022-06-11 23:24,Juice WRLD,Robbery,228629
1,2022-06-12 00:32,Calboy,Envy Me,118955
2,2022-06-12 00:36,Juice WRLD,Robbery,240050
3,2022-06-12 00:38,YNW Melly,Mixed Personalities (feat. Kanye West),131332
4,2022-06-12 00:41,Brent Faiyaz,Rehab (Winter In Paris),180303
...,...,...,...,...
78576,2023-06-12 22:34,Bryson Tiller,Let Me Explain,37730
78577,2023-06-12 22:35,Bryson Tiller,Canceled,70320
78578,2023-06-12 22:45,Ryan Trey,Nowhere to Run (feat. Bryson Tiller),146370
78579,2023-06-12 22:48,Bryson Tiller,Run Me Dry,169266


In [27]:
#quick summary of the data 
df.describe()

,endTime,artistName,trackName,msPlayed
count,78581,78581,78581,78581
unique,48238,2535,9857,28898
top,2022-09-04 16:47,Don Toliver,"Dreams, Fairytales, Fantasies (feat. Brent Fai...",0
freq,40,3829,241,728


In [128]:
#finding my top 20 most played artists
artist_count = df["artistName"].value_counts()
df["artistCount"] = df["artistName"].map(artist_count)
Fav_artists = df[["artistName", "artistCount"]]
Fav_artists = Fav_artists.drop_duplicates()
Fav_artists_sorted = Fav_artists.sort_values(by="artistCount", ascending=False)
Fav_artists_sorted  = Fav_artists_sorted.reset_index(drop=True)
Fav_artists_sorted.head(20)

,artistName,artistCount
0,Don Toliver,3829
1,A Boogie Wit da Hoodie,2383
2,Bryson Tiller,1894
3,Drake,1829
4,Lil Baby,1702
5,Lil Tjay,1701
6,Shoreline Mafia,1691
7,Summer Walker,1398
8,Metro Boomin,1357
9,Juice WRLD,1356


In [135]:
#finding average number of songs played per day 
df[["date", "time"]] = df['endTime'].str.split(' ', expand=True)
date_count = df["date"].value_counts()

songs_everyday = pd.DataFrame(df["date"])
songs_everyday["dateCount"] = df["date"].map(date_count)
songs_everyday = songs_everyday.drop_duplicates()
songs_everyday = songs_everyday.drop(0)
songs_everyday  = songs_everyday.reset_index(drop=True)
songs_everyday


,date,dateCount
0,2022-06-12,175
1,2022-06-13,246
2,2022-06-14,255
3,2022-06-15,282
4,2022-06-16,408
...,...,...
359,2023-06-08,150
360,2023-06-09,278
361,2023-06-10,236
362,2023-06-11,37


In [136]:
avg = songs_everyday["dateCount"].mean()
print(avg)
#avg = 216 songs per day, about 10 hours per day 


215.87912087912088


In [118]:
#find hours listened per month 
import re 

#new column for minutes played per song
#for each date, drop the year and date, just keep the month


new_table = pd.DataFrame(df[["msPlayed","date"]])
new_table["date"]=pd.to_datetime(new_table["date"])
new_table["month"] = new_table["date"].dt.month

new_table['msPlayed'] = new_table['msPlayed'].apply(lambda x: re.sub('[^0-9]', '', x))
new_table["msPlayed"] = new_table["msPlayed"].astype(int)
new_table['HoursPlayed'] = new_table['msPlayed'] / 3600000
monthly_totals = new_table.groupby("month")["HoursPlayed"].sum().reset_index()
new_table = new_table.merge(monthly_totals, on='month', suffixes=('', '_total'))
new_table = new_table.drop_duplicates("month")
final_table = new_table[["month","HoursPlayed_total"]]
final_table = final_table.sort_values(by="month", ascending=True)
final_table = final_table.reset_index(drop=True)
final_table

,month,HoursPlayed_total
0,1,3.158897e+08
1,2,1.640308e+02
2,3,6.177395e+10
3,4,2.711672e+10
4,5,1.223519e+02
5,6,1.843627e+02
6,7,4.938256e+10
7,8,3.894524e+07
8,9,1.338827e+02
9,10,5.902783e+10


In [131]:
#finding top 10 most played songs
#create new column = unique od artistname + trackname and count on that 
df["uniqueID"] = df["trackName"]+ " by " + df["artistName"]

song_count = df["uniqueID"].value_counts()
df["songCount"] = df["uniqueID"].map(song_count)
songs_table = df[["uniqueID", "songCount"]]
songs_table = songs_table.drop_duplicates()
songs_table = songs_table.sort_values(by="songCount", ascending=False)
songs_table  = songs_table.reset_index(drop=True)
songs_table.head(30)

,uniqueID,songCount
0,"Dreams, Fairytales, Fantasies (feat. Brent Fai...",241
1,Drugs N Hella Melodies (feat. Kali Uchis) by D...,236
2,Next To You by Bryson Tiller,210
3,California Breeze by Lil Baby,207
4,Skoompin' by ShooterGang Kony,207
5,5X by Don Toliver,205
6,Asshole by Mike Sherm,201
7,Best You Had by Don Toliver,201
8,Luv Is Dro (feat. Static Major & Bryson Tiller...,192
9,WAIT FOR U (feat. Drake & Tems) by Future,185


In [138]:
#need to export all these as csv files to put into tableau 

#table for top 20 artists
Fav_artists_sorted.to_csv('Top20Artists.csv')

#total songs listened to everyday
songs_everyday.to_csv('TotalSongsEveryday.csv')

#total hours listened per month
final_table.to_csv('HoursListenedperMonth.csv')

#top 20 songs most listened to 
songs_table.to_csv('Top20Songs.csv')

print('done')


done
